# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model  = model_LDA(lattice, atoms, positions, symmetries=false)
basis  = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-8);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.234314128853                   -0.50    7.0         
  2   -7.249981325809       -1.81       -1.40    1.0   13.1ms
  3   -7.250862155669       -3.06       -1.86    2.0   14.9ms
  4   -7.250828481256   +   -4.47       -1.81    2.0   16.3ms
  5   -7.251333821555       -3.30       -2.77    1.0   13.8ms
  6   -7.251338036207       -5.38       -3.15    2.0   16.5ms
  7   -7.251338636516       -6.22       -3.45    2.0   19.0ms
  8   -7.251338791331       -6.81       -4.11    1.0   14.7ms
  9   -7.251338797995       -8.18       -4.73    1.0   13.9ms
 10   -7.251338798577       -9.24       -5.05    2.0   17.1ms
 11   -7.251338798694       -9.94       -5.65    1.0   14.0ms
 12   -7.251338798703      -11.01       -6.03    2.0   16.3ms
 13   -7.251338798704      -12.03       -6.46    1.0   13.9ms
 14   -7.251338798705      -12.89       -6.89    2.0   15.3ms
 15   -7.

Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.08122086748336733
[ Info: Arnoldi iteration step 2: normres = 0.4466291449638523
[ Info: Arnoldi iteration step 3: normres = 0.7462757887349021
[ Info: Arnoldi iteration step 4: normres = 0.3096278601393358
[ Info: Arnoldi iteration step 5: normres = 0.6090658999315895
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (3.64e-02, 7.53e-02, 5.17e-01, 3.08e-01, 4.07e-02)
[ Info: Arnoldi iteration step 6: normres = 0.15760716929398366
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.94e-03, 1.10e-01, 7.02e-02, 7.36e-02, 1.33e-02)
[ Info: Arnoldi iteration step 7: normres = 0.07936795050467393
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.76e-04, 6.75e-03, 4.18e-03, 2.17e-02, 7.34e-02)
[ Info: Arnoldi iteration step 8: normres = 0.0805229594669775
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (6.00e-0